In [1]:
import pandas as pd
import numpy as np
import math
import os 
from sklearn import preprocessing, neighbors, model_selection

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')
pd.set_option('display.max_rows', 10)
pd.set_option('display.notebook_repr_html', True)
pd.set_option('display.max_columns', 10)


In [2]:
df = pd.read_csv("dataset-06-storms.csv.gz",low_memory=False)

In [3]:
df

,STATE__,BGN_DATE,BGN_TIME,TIME_ZONE,COUNTY,...,LONGITUDE,LATITUDE_E,LONGITUDE_,REMARKS,REFNUM
0,1.0,4/18/1950 0:00:00,130,CST,97.0,...,8812.0,3051.0,8806.0,NaN,1.0
1,1.0,4/18/1950 0:00:00,145,CST,3.0,...,8755.0,0.0,0.0,NaN,2.0
2,1.0,2/20/1951 0:00:00,1600,CST,57.0,...,8742.0,0.0,0.0,NaN,3.0
3,1.0,6/8/1951 0:00:00,900,CST,89.0,...,8626.0,0.0,0.0,NaN,4.0
4,1.0,11/15/1951 0:00:00,1500,CST,43.0,...,8642.0,0.0,0.0,NaN,5.0
...,...,...,...,...,...,...,...,...,...,...,...
902292,56.0,11/30/2011 0:00:00,10:30:00 PM,MST,7.0,...,0.0,0.0,0.0,EPISODE NARRATIVE: A strong cold front moved s...,902293.0
902293,30.0,11/10/2011 0:00:00,02:48:00 PM,MST,9.0,...,0.0,0.0,0.0,EPISODE NARRATIVE: A strong westerly flow alof...,902294.0
902294,2.0,11/8/2011 0:00:00,02:58:00 PM,AKS,213.0,...,0.0,0.0,0.0,EPISODE NARRATIVE: A 960 mb low over the south...,902295.0
902295,2.0,11/9/2011 0:00:00,10:21:00 AM,AKS,202.0,...,0.0,0.0,0.0,EPISODE NARRATIVE: A 960 mb low over the south...,902296.0


In [4]:
df['STATE__'].unique()

array([  1.,   4.,   5.,   6.,   8.,   9.,  10.,  11.,  12.,  13.,  15.,
        16.,  17.,  18.,  19.,  20.,  21.,  22.,  23.,  24.,  25.,  26.,
        27.,  28.,  29.,  30.,  31.,  32.,  33.,  34.,  35.,  36.,  37.,
        38.,  39.,  40.,  41.,  42.,  44.,  45.,  46.,  47.,  48.,  49.,
        50.,  51.,  53.,  54.,  55.,  56.,  72.,   2.,  60.,  66.,  68.,
        78.,  87.,  81.,  84.,  85.,  86.,  88.,  90.,  91.,  95.,  92.,
        93.,  94.,  83.,  89.])

In [5]:
df['FATALITIES'].unique()

array([   0.,    1.,    4.,    6.,    7.,    2.,    5.,   25.,    3.,
         10.,   14.,    9.,   11.,   23.,   22.,   21.,   50.,   29.,
         34.,   17.,    8.,   18.,   24.,   33.,   31.,   20.,   13.,
         75.,   15.,   16.,   90.,  116.,   12.,   38.,   57.,   30.,
         37.,   36.,  114.,   26.,   42.,   27.,  583.,   67.,   19.,
         32.,   99.,   74.,   49.,   46.,   44.,  158.])

In [6]:
df['PROPDMG'].unique()

array([  25. ,    2.5,  250. , ...,  164.8,  181. ,   43.5])

In [8]:
FAT = df.sort_values('FATALITIES')

FAT[-10:].sort_values('FATALITIES')

,STATE__,BGN_DATE,BGN_TIME,TIME_ZONE,COUNTY,...,LONGITUDE,LATITUDE_E,LONGITUDE_,REMARKS,REFNUM
78566,28.0,3/3/1966 0:00:00,1830,CST,49.0,...,9033.0,3221.0,9007.0,NaN,78567.0
230926,42.0,7/1/1995 0:00:00,0,EST,0.0,...,0.0,0.0,0.0,While the first ten days of July 1995 were coo...,230914.0
371111,42.0,7/4/1999 0:00:00,08:00:00 AM,EST,54.0,...,0.0,0.0,0.0,A very strong and oppressive high pressure sys...,371072.0
46308,20.0,5/25/1955 0:00:00,2245,CST,35.0,...,9709.0,3723.0,9707.0,NaN,46309.0
67883,25.0,6/9/1953 0:00:00,1425,CST,27.0,...,7210.0,4218.0,7131.0,NaN,67884.0
355127,17.0,7/28/1999 0:00:00,11:00:00 AM,CST,5.0,...,0.0,0.0,0.0,An intense heat wave began in the Chicago metr...,355098.0
148851,48.0,5/11/1953 0:00:00,1610,CST,309.0,...,9709.0,3145.0,9653.0,NaN,148852.0
68669,26.0,6/8/1953 0:00:00,1930,CST,49.0,...,8351.0,4306.0,8339.0,NaN,68670.0
862633,29.0,5/22/2011 0:00:00,04:40:00 PM,CST,97.0,...,9434.0,3703.0,9424.0,EPISODE NARRATIVE: A strong upper level trough...,862563.0
198703,17.0,7/12/1995 0:00:00,1100,CST,0.0,...,0.0,0.0,0.0,An intense heat wave affected northern Illinoi...,198690.0


In [9]:
df[['FATALITIES', 'EVTYPE', 'INJURIES', 'PROPDMG', 'CROPDMG', 'PROPDMGEXP', 'CROPDMGEXP', 'STATE', 'BGN_DATE']]

,FATALITIES,EVTYPE,INJURIES,PROPDMG,CROPDMG,PROPDMGEXP,CROPDMGEXP,STATE,BGN_DATE
0,0.0,TORNADO,15.0,25.0,0.0,K,NaN,AL,4/18/1950 0:00:00
1,0.0,TORNADO,0.0,2.5,0.0,K,NaN,AL,4/18/1950 0:00:00
2,0.0,TORNADO,2.0,25.0,0.0,K,NaN,AL,2/20/1951 0:00:00
3,0.0,TORNADO,2.0,2.5,0.0,K,NaN,AL,6/8/1951 0:00:00
4,0.0,TORNADO,2.0,2.5,0.0,K,NaN,AL,11/15/1951 0:00:00
...,...,...,...,...,...,...,...,...,...
902292,0.0,HIGH WIND,0.0,0.0,0.0,K,K,WY,11/30/2011 0:00:00
902293,0.0,HIGH WIND,0.0,0.0,0.0,K,K,MT,11/10/2011 0:00:00
902294,0.0,HIGH WIND,0.0,0.0,0.0,K,K,AK,11/8/2011 0:00:00
902295,0.0,BLIZZARD,0.0,0.0,0.0,K,K,AK,11/9/2011 0:00:00


In [10]:
df[['FATALITIES', 'EVTYPE', 'INJURIES', 'PROPDMG', 'CROPDMG', 'PROPDMGEXP', 'CROPDMGEXP', 'STATE', 'BGN_DATE']].sort_values('FATALITIES',ascending=False)

,FATALITIES,EVTYPE,INJURIES,PROPDMG,CROPDMG,PROPDMGEXP,CROPDMGEXP,STATE,BGN_DATE
198703,583.0,HEAT,0.0,0.0,0.0,NaN,NaN,IL,7/12/1995 0:00:00
862633,158.0,TORNADO,1150.0,2.8,0.0,B,K,MO,5/22/2011 0:00:00
68669,116.0,TORNADO,785.0,25.0,0.0,M,NaN,MI,6/8/1953 0:00:00
148851,114.0,TORNADO,597.0,25.0,0.0,M,NaN,TX,5/11/1953 0:00:00
355127,99.0,EXCESSIVE HEAT,0.0,0.0,0.0,NaN,NaN,IL,7/28/1999 0:00:00
...,...,...,...,...,...,...,...,...,...
302028,0.0,TSTM WIND,0.0,0.0,0.0,NaN,NaN,SC,7/4/1997 0:00:00
302029,0.0,TSTM WIND,0.0,0.0,0.0,NaN,NaN,SC,7/4/1997 0:00:00
302030,0.0,TSTM WIND,0.0,0.0,0.0,NaN,NaN,SC,7/5/1997 0:00:00
302031,0.0,TSTM WIND,0.0,0.0,0.0,NaN,NaN,SC,7/5/1997 0:00:00


In [11]:
df.loc[df['FATALITIES'] >= 1, ['FATALITIES', 'EVTYPE', 'INJURIES', 'PROPDMG', 'CROPDMG', 'PROPDMGEXP', 'CROPDMGEXP', 'STATE', 'BGN_DATE']].sort_values('FATALITIES',ascending=False)

,FATALITIES,EVTYPE,INJURIES,PROPDMG,CROPDMG,PROPDMGEXP,CROPDMGEXP,STATE,BGN_DATE
198703,583.0,HEAT,0.0,0.00,0.0,NaN,NaN,IL,7/12/1995 0:00:00
862633,158.0,TORNADO,1150.0,2.80,0.0,B,K,MO,5/22/2011 0:00:00
68669,116.0,TORNADO,785.0,25.00,0.0,M,NaN,MI,6/8/1953 0:00:00
148851,114.0,TORNADO,597.0,25.00,0.0,M,NaN,TX,5/11/1953 0:00:00
355127,99.0,EXCESSIVE HEAT,0.0,0.00,0.0,NaN,NaN,IL,7/28/1999 0:00:00
...,...,...,...,...,...,...,...,...,...
325009,1.0,TSTM WIND,0.0,35.00,0.0,K,NaN,MI,5/31/1998 0:00:00
324968,1.0,TSTM WIND,0.0,100.00,0.0,K,NaN,MI,5/31/1998 0:00:00
324952,1.0,TSTM WIND,0.0,502.70,0.0,K,NaN,MI,5/31/1998 0:00:00
324908,1.0,TSTM WIND,20.0,34.63,10.0,M,M,MI,5/31/1998 0:00:00
